In [19]:
# Imports
import pandas as pd
from dotenv import load_dotenv
import os
import csv
import sys
import re

### Data processing

In [ ]:
load_dotenv()
filepath = os.getenv('FILEPATH')
print(f"Filepath: {filepath}") 

#df_10 = pd.read_csv(filepath, nrows=10)
#print(df_10)


Filepath: data/news_cleaned_2018_02_13.csv


In [ ]:
"""
The corpus is formatted as a CSV and contains the following fields:

    id
    domain
    type
    url
    content
    scraped_at
    inserted_at
    updated_at
    title
    authors
    keywords
    meta_keywords
    meta_description
    tags
    summary
    source (opensources, nytimes, or webhose)
"""
csv.field_size_limit(10**9)
selected_columns = ["id", "domain", "type", "url", "content", "title"]

# using 100 000 rows for building the data processing for now
df = pd.read_csv(filepath, nrows=100000, encoding="utf-8", usecols=selected_columns, on_bad_lines='warn')


# Load the CSV file in chunks of 100000 rows
# chunks = []
# for chunk in pd.read_csv(filepath, usecols=selected_columns, chunksize=500000, on_bad_lines='warn', engine='python'):
#    chunks.append(chunk)
# df = pd.concat(chunks, ignore_index=True)


#for i, chunk in enumerate(df_iter):
#    df = pd.concat([df, chunk], ignore_index=True)
#    print(f"Chunk {i+1}:")
#    print(chunk.head())  # Printing first 5 rows of each chunk

100000
100000
test
   id              domain   type  \
0   2       express.co.uk  rumor   
1   6  barenakedislam.com   hate   
2   7  barenakedislam.com   hate   
3   8  barenakedislam.com   hate   
4   9  barenakedislam.com   hate   

                                                 url  \
0  https://www.express.co.uk/news/science/738402/...   
1  http://barenakedislam.com/category/donald-trum...   
2  http://barenakedislam.com/category/donald-trum...   
3  http://barenakedislam.com/2017/12/24/more-winn...   
4  http://barenakedislam.com/2017/12/25/oh-trump-...   

                                             content  \
0  Life is an illusion, at least on a quantum lev...   
1  Unfortunately, he hasn’t yet attacked her for ...   
2  The Los Angeles Police Department has been den...   
3  The White House has decided to quietly withdra...   
4  “The time has come to cut off the tongues of t...   

                                               title  
0  Is life an ILLUSION? Researchers

In [ ]:
# Cleaning data

#print(df.shape[0])
#print(df.isnull().sum())
#print(df.duplicated().sum())

# Removing duplicates and null values from speficic columns
df = df.drop_duplicates(subset=["url", "content"])
df = df.dropna(subset=["title"])

# lowercasing text
df['content'] = df['content'].str.lower()
df['title'] = df['title'].str.lower()

# removing punctuations
def remove_punctutation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['content'] = df['content'].apply(remove_punctutation)
df['title'] = df['title'].apply(remove_punctutation)

#print(df.shape[0])

#print("test")
#print(df.head())  

99969
id            0
domain        0
type       3509
url           0
content       0
title         0
dtype: int64
0
99969
test
   id              domain   type  \
0   2       express.co.uk  rumor   
1   6  barenakedislam.com   hate   
2   7  barenakedislam.com   hate   
3   8  barenakedislam.com   hate   
4   9  barenakedislam.com   hate   

                                                 url  \
0  https://www.express.co.uk/news/science/738402/...   
1  http://barenakedislam.com/category/donald-trum...   
2  http://barenakedislam.com/category/donald-trum...   
3  http://barenakedislam.com/2017/12/24/more-winn...   
4  http://barenakedislam.com/2017/12/25/oh-trump-...   

                                             content  \
0  life is an illusion at least on a quantum leve...   
1  unfortunately he hasnt yet attacked her for is...   
2  the los angeles police department has been den...   
3  the white house has decided to quietly withdra...   
4  the time has come to cut off the to

## Data processing